<a href="https://colab.research.google.com/github/mmao114/Analysis-of-basic-information-and-reviews-of-ski-resorts/blob/main/%E5%8F%A3%E3%82%B3%E3%83%9Fpca%E5%89%8D%E5%87%A6%E7%90%86%E7%B5%90%E6%9E%9C%E4%BF%AE%E6%AD%A3%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7
!pip install janome
!apt-get -y install fonts-ipafont-gothic
!pip install japanize-matplotlib

!pip install mojimoji
!pip install neologdn
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null
#!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30
Suggested packages:
  apt-xapian-index aptitude-doc-en | aptitude-doc debtags tasksel libcwidget-dev xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30
0 upgraded, 5 newly installed, 0 to remove and 49 not upgraded.
Need to get 3,838 kB of archives.
After this operation, 17.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aptitude-common all 0.8.13-3ubuntu1 [1,719 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsigc++-2.0-0v5 amd64 2.10.4-2ubuntu3 [12.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcwidget4 amd64 0.5.18-5build1 [306 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libxapian30 amd64 1

In [ ]:
import pandas as pd
import MeCab
from collections import Counter
import re

# 初始化 MeCab
mecab = MeCab.Tagger('-Ochasen')

# 定义停用词列表
stopword_list = [
    "and", "of", "the", "し", "い", "ある", "おる", "せる", "ない", "いる", "する", "の", "よう", "人", "れる",
    "なる", "それ", "そこ", "これ", "こう", "ため", "そう", "こと", "られる", "!", "#", "...", "0", "?", "ww", "♪",
    "(", ")", ".", ",", "%", ":", "/", "•", "＝", "-", "〇", "✓", "_", "この", "は", "が", "も", "を", "に", "へ", "と",
    "で", "や", "か", "だ", "よ", "ね", "な", "その", "あの", "どの", "何", "どれ", "どんな", "たい", "した", "です",
    "ます", "ません", "いる", "う", "いう", "思う", "みる", "せる", "させる", "いい", "すべて", "ただ", "しかし",
    "また", "それに", "なので", "すると", "だから", "そして", "エリア", "的", "月", "者", "事", "向け", "市", "度", "内",
    "数", "等", "年", "月", "日", "時", "分", "秒", "化", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k",
    "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "A", "B", "C", "D", "E", "F", "G",
    "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "&", "-", "！",
    "◎", "。", "、",  "まし", "でし", "から", "ので", "......", "あり", "場所", "とても", "まで", "市内",
    "スキー", "コース", "田代", "札幌", "白馬", "ませ", "でき", "さ", "…”", "【", "】", "一", "”", "《", "》",
    "「", "」", "『", "』", "〈", "〉", "☆", "★", "○", "×", "「", "」", "【", "】", "→", "←", "↑", "↓", "♪",
    "☆", "★", "※", "→", "←", "↑", "↓", "……", "ーー", "ッ", "〜", "　", "・", "—", "ーー", "ー", "、", "！",
    "？", "◇", "◆", "○", "●", "△", "▲", "□", "■", "×", "◎", "◯"
]

# 上传并读取 Excel 文件，注意修改文件路径
file_path = '/content/サイド情報.xlsx'  # 替换为你的 Excel 文件路径
df = pd.read_excel(file_path)

# 只读取第二列，假设列索引从0开始，第二列就是索引1
text_data = df.iloc[:, 1].dropna().astype(str).str.cat(sep=' ')  # 将所有文本合并为一个字符串

# 使用 MeCab 分词并获取词性
parsed_text = mecab.parse(text_data)

# 过滤条件：排除副词、标点符号、单个字符
words = []
for line in parsed_text.splitlines():
    if "\t" in line:  # 检查是否包含分词结果
        word, feature = line.split("\t")[0], line.split("\t")[3]
        # 检查是否是名词，并且不在停用词列表中，同时长度大于1（排除单个字符）
        if "名詞" in feature and word not in stopword_list and len(word) > 1:
            words.append(word)

# 计算词频
word_count = Counter(words)

# 获取词频最高的前10个词组
top_10_words = word_count.most_common(5)

# 显示结果
print("Top 10 frequent phrases:")
for word, freq in top_10_words:
    print(f'{word}: {freq}')


Top 10 frequent phrases:
ゲレンデ: 47
上級: 29
初心者: 27
山頂: 19
標高: 15


In [ ]:
import pandas as pd
import MeCab
from collections import Counter
import os

# 初始化 MeCab
mecab = MeCab.Tagger('-Ochasen')

# 定义你之前得到的Top 10词组
top_10_words = ['リフト', 'ゲレンデ', '最高', '初心者', 'ゴンドラ', '景色', '時間', '子供', '上級', 'レンタル']  # 替换为你的实际Top 10词组

# 定义处理单个文件的函数，读取第二列并统计Top 10词组的词频
def process_excel(file_path):
    # 读取 Excel 文件的第二列
    df = pd.read_excel(file_path)
    text_data = df.iloc[:, 1].dropna().astype(str).str.cat(sep=' ')  # 将所有文本合并为一个字符串

    # 使用 MeCab 分词
    parsed_text = mecab.parse(text_data)

    # 统计词频
    words = []
    for line in parsed_text.splitlines():
        if "\t" in line:
            word, feature = line.split("\t")[0], line.split("\t")[3]
            if "名詞" in feature and len(word) > 1:  # 保留名詞且长度大于1的词
                words.append(word)

    # 计算Top 10词组在该文件中的频率
    word_count = Counter(words)
    total_reviews = len(df.iloc[:, 1].dropna())  # 获取评论的数量
    return [word_count.get(word, 0) / total_reviews if total_reviews > 0 else 0 for word in top_10_words]

# 定义函数遍历文件夹，处理所有Excel文件
def process_folder(folder_path):
    # 创建一个用于保存结果的列表，每个元素是一个字典，表示一行数据
    all_data = []

    # 遍历文件夹中的所有Excel文件
    for filename in os.listdir(folder_path):
        if filename.endswith('.xlsx'):  # 确保只处理Excel文件
            file_path = os.path.join(folder_path, filename)
            word_frequencies = process_excel(file_path)

            # 去掉文件名的后缀，只保留文件名
            file_base_name = os.path.splitext(filename)[0]

            # 将文件名与词频结果一起添加到列表中
            row_data = {'r': file_base_name}  # 将文件名（无后缀）存储到字典中
            for i, word in enumerate(top_10_words):
                row_data[word] = word_frequencies[i]  # 添加词频数据到字典
            all_data.append(row_data)  # 将字典添加到列表中

    # 创建 DataFrame
    result_df = pd.DataFrame(all_data)

    # 返回最终的结果DataFrame
    return result_df

# 读取文件夹路径
folder_path = '/content/Untitled Folder'  # 替换为你上传的文件夹路径

# 处理文件夹中的所有Excel文件
result_df = process_folder(folder_path)

# 显示结果
print(result_df)

# 将结果导出为 Excel 文件
result_df.to_excel('word_frequencies_homepage.xlsx', index=False)


                  r       リフト      ゲレンデ        最高       初心者      ゴンドラ  \
0       赤井村キロロリゾート  0.110169  0.068644  0.115254  0.042373  0.076271   
1      札幌滝野スノーワールド  0.132565  0.077810  0.031700  0.103746  0.002882   
2        新潟かぐらスキー場  0.098246  0.107018  0.131579  0.082456  0.059649   
3        長野栂池高原スキー場  0.132155  0.207067  0.164664  0.144876  0.105300   
4      長野白馬八方尾根スキー場  0.223699  0.171096  0.141750  0.101329  0.152824   
5            長野野沢温泉  0.102687  0.201536  0.182342  0.113244  0.189060   
6          札幌国際スキー場  0.108664  0.054332  0.086637  0.114537  0.106461   
7   長野白馬岩岳スノーフィールド  0.085944  0.104418  0.193574  0.032932  0.149398   
8        長野白馬五竜スキー場  0.166057  0.185808  0.122165  0.181419  0.158010   
9       札幌ばんけいスキー場  0.096822  0.059128  0.054693  0.127125  0.002956   
10         新潟苗場スキー場  0.167206  0.144552  0.083064  0.099245  0.107875   
11        新潟湯沢スキー場   0.119492  0.090678  0.069492  0.138983  0.078814   
12       札幌サッポロテイネ  0.104193  0.068615  0.078780  